# README

This script calculates model scores for the neuronal differentiation classification model described in Jerber et al, 2020.

Note that this model was trained on Z-score normalised iPSC RNA-seq data from one project (the HipSci project). This has the following implications:

- The model cannot be applied at all to a single sample (for which the Z-score is undefined), and will not produce  stable predictions for small numbers of samples.
- While the Z-score normalization was intended to improve portability of the model across iPSC RNA-seq samples from other projects, this is untested. In particular, the model score threshold applied in the paper to generate high-precision predictions may not be well calibrated in another dataset. Therefore, a more appropriate application of the model score in that context would be to rank and prioritize cell lines that are the most likely to differentiate well. 


In [31]:
import pandas as pd
import numpy as np
import os

In [32]:
# specify input files
t_file = './example_expression_data.tsv'
coefficient_file = './model_coefficients.tsv'
out_file = './example_output_file.tsv'

In [33]:
# load bulk RNA-seq data

t_df = pd.read_csv(t_file, sep='\t', index_col=0)

# load model coefficients

coef_df = pd.read_csv(coefficient_file, sep='\t', index_col=0)

In [34]:
# read gene expression table and check for missing data

feature_list = t_df.index.intersection(coef_df.index)

model_feature_list = [x for x in coef_df.index if x!='intercept']

# check for model features that are not in the input transcript data
missing_features = [x for x in model_feature_list if x not in feature_list]

if len(missing_features)>0:
    print('Warning: {} genes missing from the input gene expression table:'.format(len(missing_features)))
    for gene in missing_features:
        print(gene)


In [35]:
# get model coefficients in appropriate form for NumPy

coefficients = coef_df.loc[feature_list, 'coefficient'].values.reshape((1, -1))

intercept = coef_df.loc['intercept','coefficient']

In [36]:
# transform input expression data into z-score normalized NumPy array

t_df = t_df.loc[feature_list, :]

# apply z-score transformation to the data
t_df = t_df.apply(lambda x: (x-x.mean())/x.std(), axis=1)

X = t_df.transpose().values

In [37]:
# compute model scores for logistic regression model

linear_scores = list((np.matmul(coefficients, X.transpose()) + intercept)[0])
prob_scores = [1/(1+np.exp(-x)) for x in linear_scores]

out_df = pd.DataFrame({'cell_line_id':t_df.columns,
                       'model_score':prob_scores})

out_df

,cell_line_id,model_score
0,HPSI0114i-bezi_1,0.370912
1,HPSI0114i-bezi_3,0.943307
2,HPSI0114i-eipl_1,0.000856
3,HPSI0114i-eipl_3,0.000060
4,HPSI0114i-fikt_3,0.764334
5,HPSI0114i-iisa_1,0.866401
6,HPSI0114i-iisa_3,0.747887
7,HPSI0114i-joxm_1,0.849625
8,HPSI0114i-kolf_2,0.764440
9,HPSI0114i-kolf_3,0.869390


In [38]:
# output to file

out_df.to_csv(out_file, sep='\t', index=False)